## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-03-13-09-00 +0000,wsj,PayPal Replaces CEO as it Flags Lower Earnings,https://www.wsj.com/business/earnings/paypal-n...
1,2026-02-03-13-06-05 +0000,nyt,Son of Norway’s Crown Princess Goes on Trial O...,https://www.nytimes.com/2026/02/03/world/europ...
2,2026-02-03-13-04-00 +0000,bbc,Rape trial begins for son of Norway's crown pr...,https://www.bbc.com/news/articles/ckgn8dnxjgwo...
3,2026-02-03-13-02-11 +0000,bbc,No defect found in switch of jet grounded by A...,https://www.bbc.com/news/articles/c8e5zwndddyo...
4,2026-02-03-13-02-00 +0000,wsj,Russia Likely to Struggle to Place Oil If Indi...,https://www.wsj.com/finance/commodities-future...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2471/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
281,trump,43
61,epstein,24
333,billion,13
695,deal,12
156,ice,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
211,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...,109
152,2026-02-03-02-16-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...,96
291,2026-02-02-18-39-33 +0000,nypost,"Trump says Jeffrey Epstein, Michael Wolff ‘con...",https://nypost.com/2026/02/02/us-news/trump-sa...,93
317,2026-02-02-17-14-00 +0000,wsj,President Trump’s choice of Kevin Warsh as cha...,https://www.wsj.com/economy/central-banking/wh...,89
197,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...,87


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
211,109,2026-02-02-22-44-00 +0000,wsj,Trump demanded Monday that House lawmakers bac...,https://www.wsj.com/politics/policy/trump-push...
152,85,2026-02-03-02-16-00 +0000,wsj,Bill and Hillary Clinton made a new offer to g...,https://www.wsj.com/politics/policy/bill-hilla...
183,55,2026-02-03-00-07-20 +0000,nypost,Norway’s Epstein-tainted princess in yet anoth...,https://nypost.com/2026/02/02/world-news/norwa...
317,46,2026-02-02-17-14-00 +0000,wsj,President Trump’s choice of Kevin Warsh as cha...,https://www.wsj.com/economy/central-banking/wh...
197,44,2026-02-02-23-08-00 +0000,wsj,The Trump administration is preparing to devel...,https://www.wsj.com/politics/policy/trump-admi...
194,43,2026-02-02-23-23-39 +0000,cbc,Body cameras being immediately issued to ICE a...,https://www.cbc.ca/news/world/minneapolis-agen...
278,40,2026-02-02-19-19-00 +0000,wsj,U.S. Will Cut Tariffs on India to 18% in Trade...,https://www.wsj.com/politics/policy/u-s-will-c...
258,38,2026-02-02-20-20-26 +0000,wapo,Rafah crossing between Gaza and Egypt reopens ...,https://www.washingtonpost.com/world/2026/02/0...
106,38,2026-02-03-08-30-23 +0000,nypost,Savannah Guthrie’s sister enjoyed dinner with ...,https://nypost.com/2026/02/03/us-news/savannah...
315,35,2026-02-02-17-14-38 +0000,cbc,"Powerful men from politics, business and enter...",https://www.cbc.ca/news/world/epstein-files-no...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
